In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, StratifiedShuffleSplit,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, average_precision_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.utils import Bunch
from scipy.stats import randint, loguniform

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import warnings
from sklearn.model_selection import KFold, cross_val_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('silver_stock_info.csv')

In [35]:
modelDF = df[['open', 'high', 'low', 'close', 'volume','movement']]


CONFIG = {
    "test_size": 0.30,
    "random_state": 18,
    "cv_folds": 10,
}

In [12]:
from sklearn.dummy import DummyClassifier

majority_clf = DummyClassifier(strategy="most_frequent")
majority_clf.fit(features, target)

y_pred_majority = majority_clf.predict(features)

print("Majority Classifier Accuracy:", accuracy_score(target, y_pred_majority))
print("Classification Report:\n", classification_report(target, y_pred_majority, zero_division=0))

Majority Classifier Accuracy: 0.624
Classification Report:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77       156
           1       0.00      0.00      0.00        94

    accuracy                           0.62       250
   macro avg       0.31      0.50      0.38       250
weighted avg       0.39      0.62      0.48       250



  feature            VIF
0    open   85279.549780
1    high  110529.864389
2     low  101937.945095
3   close  112544.053811
4  volume       6.048307


In [ ]:
numeric_cols = modelDF.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in modelDF.columns if c not in numeric_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore",sparse_output=False), categorical_cols)
    ],
    remainder="drop"
)
preprocess.set_output(transform="pandas")

transformed_X = preprocess.fit_transform(modelDF)

target = transformed_X['num__movement']
features = transformed_X.drop(columns=['num__movement'])


X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=.30, random_state=18, stratify=target)




cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=18)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Numeric cols: {len(numeric_cols)}, Categorical cols: {len(categorical_cols)}")
transformed_X
vif_data = pd.DataFrame()
features

Train shape: (175, 5), Test shape: (75, 5)
Numeric cols: 6, Categorical cols: 0


,num__open,num__high,num__low,num__close,num__volume
0,1.568035,1.802701,1.504439,1.714516,0.201621
1,1.665034,1.767177,1.684300,1.740995,0.071125
2,1.989996,2.041451,1.496142,1.460808,1.713295
3,2.027938,2.041131,1.942091,1.971610,-0.191770
4,1.935835,2.019048,1.947424,2.023029,-0.406593
...,...,...,...,...,...
245,-0.496786,-0.582555,-0.474919,-0.509122,-0.826218
246,-0.491278,-0.566873,-0.420102,-0.483567,-0.395570
247,-0.438648,-0.545750,-0.432251,-0.546994,-0.851786
248,-0.612756,-0.642722,-0.519218,-0.549457,-1.022024


In [44]:


tree_clf = DecisionTreeClassifier(
    random_state=42,
    max_depth=5,
    min_samples_leaf=50,
    class_weight="balanced"
)

pipe = Pipeline([("prep", preprocess), ("clf", tree_clf)])


In [45]:

vif_data = pd.DataFrame()
vif_data["feature"] = features.columns
vif_data["VIF"] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]
print(vif_data)


       feature         VIF
0    num__open  296.863598
1    num__high  384.241467
2     num__low  401.466941
3   num__close  387.364280
4  num__volume    1.685159
